In [ ]:
import pymongo

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
tweet_collection = db["dataset"]
user_collection = db["users"]

Récupération du dernier tweet de la base de données pour pouvoir par la suite estimer l'âge d'un compte utilisateur à partir de cette date et non celle d'aujourd'hui pour éviter d'avoir des valeurs énormes, car les tweets sont figés.

In [ ]:
pipeline = [
    {
        "$sort" : {
            "created_date" : -1
        }
    },
    {
        "$limit": 1
    }
]
last_tweet_published = tweet_collection.aggregate(pipeline)
last_tweet_published = list(last_tweet_published)[0]
last_tweet_published

In [ ]:
users = tweet_collection.aggregate([
    {
        "$group": {
            "_id": "$user.id",
            "tweets": { "$push": "$$ROOT" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "user_id": "$_id",
            "tweets": 1
        }
    },
    {
        "$sort" : {
            "tweets.created_date" : 1
        }
    },
    {
        "$addFields": {
            "verified": {"$last": "$tweets.user.verified"},
            "statuses_count": {"$last": "$tweets.user.statuses_count"},
            "favourites_count": {"$last": "$tweets.user.favourites_count"},
            "age_account": {
                "$divide": [
                  {
                    "$subtract": [last_tweet_published['created_date'], {"$last": "$tweets.user.created_at"}]
                  },
                  86400000 # nombre de millisecondes dans une journée
                ]
            },
            "last_tweet_published_id": {"$last": "$tweets._id"},
        }
    },
    {
        "$addFields": {
            "avg_tweet_length": {
                "$avg": {
                    "$map": {
                        "input": "$tweets",
                        "as": "tweet",
                        "in": { "$strLenCP": "$$tweet.text" }
                    }
                }
            },
            "tweet_frequency": { "$size": "$tweets" },
        }
    },
    {
        "$project": {
            "user_id": 1,
            "tweet_ids": "$tweets._id",
            "avg_tweet_length": 1,
            "tweet_frequency": 1,
            "last_tweet_published_id": 1,
            "verified": 1,
            "statuses_count": 1,
            "favourites_count": 1,
            "ratio_friends_followers": 1,
            "age_account": 1,
            "_id": 0
        }
    }
])

user_collection.insert_many(list(users))



In [ ]:
pipeline = [
    {"$unwind": "$tweet_ids"},
    {"$group": {"_id": "$user_id", "tweet_ids_count": {"$sum": 1}}},
    {"$sort":{"tweet_ids_count":-1}}
]

# Execute the aggregation pipeline
result = user_collection.aggregate(pipeline)

# Print the results
for doc in result:
    print("user_id: {}, tweet_ids_count: {}".format(doc["_id"], doc["tweet_ids_count"]))